In [2]:
import pandas as pd
import numpy as np
import jieba
from jieba.analyse import *
from collections import Counter
import time
import os
mypath=os.getcwd()
mypath

'e:\\Python\\python_code\\KnowledgeMeme\\SAOandKM'

# 1.KM_A的抽取
- 对摘要进行分词
- 两年一个时间段，对时间进行切片
- 分好的词与词典对比，不在词典中就删除
- 每一个词对应一个知识元
- 通过TF-idf计算出每个时间段的知识元的权重
- 可视化展示，随着时间的变化知识元的变化情况

In [4]:
Dir=mypath + "\dictionary\Dir.txt" #词典，60503条
Fpath=mypath + "\dictionary\StopWords.txt" #停用词，4652条

### 1.1.划分时间段

In [4]:
labels=['2000-2001','2002-2003','2004-2005','2006-2007','2008-2009','2010-2011','2012-2013','2014-2015','2016-2017','2018-2019']

#划分时间段
def Period(LISdata):
    Summary=LISdata[['年','题名','摘要']]
    Summary.sort_values("年",inplace=True)
    labels1=labels
    Summary['时间段']=pd.cut(Summary['年'],10,labels=labels1)
    return Summary

In [5]:
LISdata=pd.read_csv(mypath+ '\data\LIS_PAPERS.csv')
ZYFC=Period(LISdata)
ZYFC.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,年,题名,摘要,时间段
8697,2000,图书馆知识管理管见,"知识经济时代的知识管理可分为宏观、中观和微观 3个层次。图书馆知识管理属微观知识管理 ,其主...",2000-2001
5455,2000,可持续发展战略对21世纪图书馆发展的启示,"结合可持续发展战略的主要思想 ,论述了知识经济时代图书馆走可持续发展之路的重要意义和必然性 ...",2000-2001
2134,2000,试论数字作品的知识产权保护,"作为信息社会发展原动力的数字技术的不断发展和普及 ,导致了一系列数字作品的出现 ,进而给现行...",2000-2001
3290,2000,国产图书馆自动化集成系统比较研究,"文章选取了四种国产图书馆自动化集成系统 ,从它们的系统功能、技术特点以及数据处理几方面进行比...",2000-2001
1569,2000,企业反竞争情报研究,"The concept,content and methods of defensive c...",2000-2001


### 1.2.分词

In [9]:
#分词
def Participle(Summary):
    Summary['摘要']=Summary['摘要'].astype(str)
    Fpath1=Fpath
    Dir1=Dir
    stopwords = [line.strip() for line in open(Fpath1, 'r', encoding='utf-8').readlines()] 
    jieba.load_userdict(Dir1)
    Summary['摘要_分词'] = Summary['摘要'].apply(lambda y: ",".join([z for z in list(jieba.cut(y)) if z not in stopwords]))
    return Summary

In [10]:
Pa=Participle(ZYFC)
Pa.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\2\jieba.cache
Loading model cost 2.172 seconds.
Prefix dict has been built successfully.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,年,题名,摘要,时间段,摘要_分词
8697,2000,图书馆知识管理管见,"知识经济时代的知识管理可分为宏观、中观和微观 3个层次。图书馆知识管理属微观知识管理 ,其主...",2000-2001,"知识经济时代,知识管理,分为,宏观,中观,微观, ,层次,图书馆知识管理,属,微观,知识管理..."
5455,2000,可持续发展战略对21世纪图书馆发展的启示,"结合可持续发展战略的主要思想 ,论述了知识经济时代图书馆走可持续发展之路的重要意义和必然性 ...",2000-2001,"可持续发展,战略,思想, ,论述,知识经济时代,图书馆,走,可持续发展,之路,意义,必然性,..."
2134,2000,试论数字作品的知识产权保护,"作为信息社会发展原动力的数字技术的不断发展和普及 ,导致了一系列数字作品的出现 ,进而给现行...",2000-2001,"信息社会,发展,原动力,数字技术,发展,普及, ,导致,一系列,数字作品, ,现行,知识产权..."
3290,2000,国产图书馆自动化集成系统比较研究,"文章选取了四种国产图书馆自动化集成系统 ,从它们的系统功能、技术特点以及数据处理几方面进行比...",2000-2001,"文章,选取,四种,国产,图书馆自动化集成系统, ,系统功能,技术,数据处理,比较分析, ,概..."
1569,2000,企业反竞争情报研究,"The concept,content and methods of defensive c...",2000-2001,"The, ,concept,content, , ,methods, , ,defensiv..."


### 1.3.按时间段切片

In [11]:
#切片
def ab(df):
    return' '.join(df.values)

Summary3 = Pa.groupby(['时间段'])['摘要_分词'].apply(ab)
Summary3

时间段
2000-2001    知识经济时代,知识管理,分为,宏观,中观,微观, ,层次,图书馆知识管理,属,微观,知识管理...
2002-2003    论述,数字图书馆,传统图书馆,关系,预测,数字图书馆,发展方向,指出,数字图书馆建设,中应 ...
2004-2005    文章,分析,大学生,信息素质教育,现实意义,主要内容, ,教育,途径,探讨 分析,供应链,供...
2006-2007    本文,虚拟参考咨询服务,动力系统,模式,分析,图书馆,效用,最大化,虚拟参考咨询,动力源,技...
2008-2009    本文,综合,学术界,看法,基础,提出,网络,信息服务质量,指标,信息服务网,站,对象,实证研...
2010-2011    发表,2000,2008,年间,代表,引文分析,研究进展,情报学文献,评述,勾勒,出,引文分...
2012-2013    面对,信息环境,用户,获取信息,手段,变化,图书馆员,致力于,利用,现代化,信息技术,读者,...
2014-2015    本文,构建,复杂网络,模块化,模型,引入,中心性,度量,集聚,系数,两类,系统化,社会,网络...
2016-2017    知识图谱,快速,兴起,发展,跨学科研究,领域,国内外,学科,间,发展轨迹,研究重点,总体,差...
2018-2019    目的, ,意义,大数据,背景,构建,合适,用户行为模型,海量,行为日志,数据,提供,个性化服...
Name: 摘要_分词, dtype: object

### 1.4.知识元遍历词典

In [14]:
#知识元遍历词典
def Traverse(Summary3):     #Summary3是按时间切片后的数据
    Dir2=Dir    
    f = open(Dir2,encoding='utf-8')
    Dir2=f.read().split("\n")
    key_words =[] 
    for i in Summary3 :
        LI = i.split(",")
        key_words.append(LI)
    for j in key_words :
        for x in j[:] :
            if x not in Dir2 :
                j.remove(x)
    return key_words

In [15]:
traverse=Traverse(Summary3)

In [1]:
# traverse

### 1.5.知识元权重随时间变化
- 取出TFidf前1000的关键词作为KM_A

In [19]:
#知识元权重随时间变化
def TFidfWeights(key_words):
    key_words3 =[] 
    for i in key_words :
        words3=jieba.analyse.extract_tags(str(i), topK=1000)#返回tfidf值排前1000的关键词
        key_words3.append(words3)
        # print(words3)
    key_words3[0]
    key_words_df4 = pd.DataFrame(key_words3).T
    key_words_df4.columns=labels
    return key_words_df4

In [21]:
TFdf=TFidfWeights(traverse)

# TFdf.to_csv(mypath+ '\generated_data\KM_A(TFIDF1000).csv')

In [22]:
TFdf.head(5)

,2000-2001,2002-2003,2004-2005,2006-2007,2008-2009,2010-2011,2012-2013,2014-2015,2016-2017,2018-2019
0,图书馆,图书馆,图书馆,图书馆,图书馆,图书馆,图书馆,图书馆,图书馆,图书馆
1,数字图书馆,数字图书馆,分析,分析,分析,分析,分析,分析,高校图书馆,智慧图书馆
2,知识管理,分析,探讨,高校图书馆,高校图书馆,研究,高校图书馆,高校图书馆,分析,文章
3,分析,高校图书馆,我国,研究,研究,高校图书馆,研究,研究,构建,高校图书馆
4,参考文献,探讨,研究,探讨,我国,公共图书馆,构建,大数据,大数据,用户


# 2.KM_B的抽取

### 2.1.从SAO里面抽取KM_B
- 1.清洗
- 2.~~Subject和Object挑选词频>n(这里设置为3),保留；否则，删除~~
- 3.和词典对比，Subject或Action或OBject在词典，保留；否则，删除；~~再去重~~

In [6]:
#SAO清洗
def SAO_clean(WJCD):       #WJCD为要导入的数据
        #抽取
        print('正在清洗SAO数据......')
        Abstract=[]
        for i in range(len(WJCD)):
            pa_p=WJCD.iloc[i]
            pa_citedpaper=pa_p[2]
            # print('第',i,'篇文章的sao：',pa_citedpaper)
            pa_citedpaper_1 = pa_citedpaper.split("$")
            # print(len(pa_citedpaper_1))
            for idx in range(0, len(pa_citedpaper_1)):
                Abstract.append([pa_citedpaper_1[idx]])
        print('抽取成功！')
        #分离
        Abstract_clean = []
        for i in Abstract :
            for j in i :
                # print(j)
                # gjc1 = j.split("#")
                gjc1=j.split('#')
                # print(gjc1)
                Abstract_clean.append(gjc1)
        print('分离成功！')
        #去重
        Abstract_clean = [i for i in Abstract_clean if i !=[' ']]
        print('去重成功！')
        return Abstract_clean

# #筛选出高频词
# def HighFrequencyWords(SAOlist,WF):  #SAOlist为清洗后的SAO,WF为输入的词频率
#     print('正在筛选SAO高频词......')
#     Subject=[]
#     Object=[]
#     for i in range(0,len(SAOlist)) :        #提取Subject和Object高频词
#         Subject.append(SAOlist[i][0])
#         Object.append(SAOlist[i][2])
#     # print(Subject)
#     cs = Counter(Subject)
#     co = Counter(Object)
#     print('词频统计成功！')
#     f = open(Fpath)
#     stopwords=f.read().split("\n")
    
#     for sw in stopwords:                    #去除停用词
#         del cs[sw]
#         del co[sw]
#     print('去除停用词成功！')
#     Sub=[]
#     Obj=[]
#     for k,v in cs.items():
#             # print(k, '=', v)
#             if v >WF :
#                 Sub.append(k)
#     for k,v in co.items():
#             # print(k, '=', v)
#             if v >WF :
#                 Obj.append(k)
#     HighSO=[]            
#     for i in range(0,len(SAOlist)) :
#         if SAOlist[i][0] in Sub or SAOlist[i][2] in Obj :  #Subject或Object出现频率高就保留
#             HighSO.append(SAOlist[i])
#     print('筛选出词频大于{name}的词条成功！'.format(name=WF))        
#     return HighSO

#去除词典中没有的词
def NotInDir(SAOlist):     #SAOlist为清理后的三元组
        print('正在遍历词典......')
        f = open(mypath + "\dictionary\Dir.txt",encoding='utf-8')
        Dir3=f.read().split("\n")    #Dir为词典
        New_SAO=[]
        for i in range(0,len(SAOlist)) :
            # print(SAO[dr])
            if (SAOlist[i][0] in Dir3) and (SAOlist[i][2] in Dir3) : #Subject和Object在词典中就保留
                New_SAO.append(SAOlist[i])
        print('词典遍历成功！共有{n}条知识基因'.format(n=len(New_SAO)))   
        #去重
#         New_SAO_clean = []
#         for id in New_SAO:
#             if id not in New_SAO_clean:
#                 New_SAO_clean.append(id)
#         print('去重成功！共有{n}条知识基因'.format(n=len(New_SAO_clean)))
        return New_SAO

##### 2.1.1.摘要SAO

In [30]:
Abstract_sao=pd.read_excel(mypath + "\data\Abstract_sao.xlsx",sheet_name='Sheet1')
ABS_SAO=NotInDir(SAO_clean(Abstract_sao))
#注：这里的摘要SAO只用‘或’和词典遍历，没有去重，没有筛选高频词；下同

正在清洗SAO数据......
抽取成功！
分离成功！
去重成功！
正在遍历词典......
词典遍历成功！共有18895条知识基因


In [38]:
# pd.DataFrame(ABS_SAO, columns=['subject', 'action', 'object']).to_csv(mypath + '\generated_data\KM_B(摘要SAO).csv')

##### 2.1.2.参考SAO

In [35]:
Ref_sao=pd.read_csv(mypath + "\data\paper_referenced_TripleExtraction.csv")
REF_SAO=NotInDir(SAO_clean(Ref_sao))

正在清洗SAO数据......
抽取成功！
分离成功！
去重成功！
正在遍历词典......
词典遍历成功！共有9016条知识基因


In [37]:
# pd.DataFrame(REF_SAO, columns=['Rsubject', 'Raction', 'Robject']).to_csv(mypath + '\generated_data\参考SAO.csv')

##### 2.1.3.引用SAO

In [6]:
Cit_sao=pd.read_excel(mypath + "\data\paper_cited_TripleExtraction.xlsx",sheet_name='paper_cited_TripleExtraction')
CIT_SAO=NotInDir(SAO_clean(Cit_sao))

正在清洗SAO数据......
抽取成功！
分离成功！
去重成功！
正在遍历词典......
词典遍历成功！共有22705条知识基因


In [50]:
# pd.DataFrame(CIT_SAO, columns=['Csubject', 'Caction', 'Cobject','none']).to_csv(mypath + '\generated_data\引文SAO.csv')

# 3.KM_Final的识别
- KM_A(TFIDF1000)和KM_B(摘要SAO)映射
- 如果Subject或Action或Object在KM_A中，保留；否则，删除。
- 保留下来的就是最终的知识基因

In [40]:
# TFIDF1000转化为列表
from tkinter import _flatten

tfidf1000tolist=list(set(list(_flatten(TFdf.values.tolist()))))

In [42]:
tfidf1000tolist[10:20]

['摘要', '索引', '信息重组', '图书馆社会化', '知识产权保护', '加权', '读者服务工作', '组织方法', '出版商', '效应']

In [44]:
KM_Final=[]
for i in range(len(ABS_SAO)) :
    if (ABS_SAO[i][0] in tfidf1000tolist) or (ABS_SAO[i][1] in tfidf1000tolist) or (ABS_SAO[i][2] in tfidf1000tolist)  :
        KM_Final.append(ABS_SAO[i])
        
print('最终的摘要知识基因有{}条'.format(len(KM_Final)))

最终的摘要知识基因有17149条


In [46]:
# pd.DataFrame(KM_Final, columns=['subject', 'action', 'object']).to_csv(mypath + '\generated_data\KM_Final.csv')